# Calculating Temporal Averages with GeoCAT-comp vs Xarray

With temporally large datasets, computing seasonal and annual averages are a great ways to summarize the data and make it easier to manage and understand. You may want to take hourly, daily, or monthly data and compute seasonal (3 month time spans) or annual averages.

## Challenges
When using data that has a daily or finer resolution (e.g. hourly), calculating an annual average is simple. Every day and hour has the same length, so an unweighted average will work.

But when using data that is monthly, things can get a bit tricky. Not every month is created equal. February has 28 or 29 days and March has 31 days. Since monthly data has one value for each month, those points can't be averaged in the usual way. A weighted average is needed.

While it is tempting to quickly compute monthly to annual averages with `Xarray`'s `resample` or `groupby` functions, we need to be careful to specify the weights. Unfortunately, `Xarray` doesn't support weighted `resample` or `groupby` at the time this post was created.

Below is a plot showing the difference between computing an annual average from monthly data using the incorrect unweighted average and the correct weighted average.

# ADD PLOT

### Imports

In [ ]:
import numpy as np
import xarray as xr
import nc_time_axis
import geocat.comp as gc
import zarr

### Helper function to make all of the plots the same way but with different data

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import geocat.viz as gv

def custom_plot(data, title):
    # Generate figure (set its size (width, height) in inches)
    plt.figure(figsize=(14, 7))

    # Generate axes, using Cartopy
    projection = ccrs.PlateCarree()
    ax = plt.axes(projection=projection)

    # Use global map and draw coastlines
    ax.set_global()
    ax.coastlines()

    # Contourf-plot data (for filled contours)
    p = data.plot.contourf(ax=ax, vmin=-10, vmax=35, levels=16, cmap='inferno',
                        add_colorbar=False, transform=projection, extend='neither')

    # Add horizontal colorbar
    cbar = plt.colorbar(p, orientation='horizontal', shrink=0.5)
    cbar.ax.tick_params(labelsize=14)
    cbar.set_ticks(np.linspace(-10, 35, 14))

    # Use geocat.viz.util convenience function to set axes tick values
    gv.set_axes_limits_and_ticks(ax,
                                 xticks=np.linspace(-180, 180, 13),
                                 yticks=np.linspace(-90, 90, 7))

    # Use geocat.viz.util convenience function to make plots look like NCL plots by using latitude, longitude tick labels
    gv.add_lat_lon_ticklabels(ax)

    # Use geocat.viz.util convenience function to add minor and major tick lines
    gv.add_major_minor_ticks(ax, labelsize=12)

    # Use geocat.viz.util convenience function to add titles to left and right of the plot axis.
    gv.set_titles_and_labels(ax, maintitle=title,
                             lefttitle=data.long_name, lefttitlefontsize=16,
                             righttitle=data.units, righttitlefontsize=16,
                             xlabel="", ylabel="")

    # Show the plot
    plt.show()

### Read in and format data

In [ ]:
ds = xr.open_zarr(
    's3://ncar-cesm2-lens/ocn/monthly/cesm2LE-ssp370-cmip6-TEMP.zarr',
    storage_options={'anon': True},
)
# Get the first 5 years of data to reduce computation time
ds_first_five_years = ds.sel(time=slice('2015', '2019')).isel(member_id=1)

ds_first_five_years

### The incorrect way to compute seasonal averages from monthly data

It's easy to compute and unweighted average using `xarray` functionality; however, this generates inaccurate results. Here is what the ***incorrect*** way of doing this looks like.

In [ ]:
seasonal_average_weighted_incorrectly = ds_first_five_years.resample(time='QS-DEC').mean().groupby('time.month').mean()
seasonal_average_weighted_incorrectly

In [ ]:
custom_plot(seasonal_average_weighted_incorrectly.TEMP.isel(z_t=0, month=0), "Incorrect Spring Average Temps")

### The correct way to compute seasonal averages with xarray
Using GeoCAT's climatology average, we can calculate the average potential temperature for each season during the 5 years our data covers. We do need to format the data before using it. `climatology_average` requires that the datetime objects for the time dimension are evenly spaced. We can use `xarray.DataArray.resample` to make sure the monthly data has dates at the start of each month rather than in the center of the months.

In [ ]:
# What the time dimension looks like before resampling
ds_first_five_years['time']

In [ ]:
# What the time dimesnion looks like after resampling. The call to mean describes how to 
# agregate the data, but we know that we only have one data point per month. We are just
# changing the day for the datetime objects.
ds_resample = ds_first_five_years.resample(time='MS').mean()
ds_resample

In [ ]:
seasonal_average_weighted_correctly = gc.climatology_average(ds_resample, 'season')
seasonal_average_weighted_correctly

In [ ]:
custom_plot(seasonal_average_weighted_correctly.TEMP.isel(z_t=0, time=0), 'Correct Sping Average Temps')

## So what's the difference?
It is hard to see the difference between the correct and incorrect ways of caluclating the seasonal averages. If we plot the difference between the two results, the computational errors become easier to see.

In [ ]:
diff = seasonal_average_weighted_correctly - seasonal_average_weighted_incorrectly
diff.max()